In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
#file path
file_path = "Small.xlsx"

In [3]:
def read_excel_data(filename, sheet_name):
    data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
    values = data.values
    return values

In [4]:
def cal_OD():
	O = []
	D = []
	for i in range(nodeNum):
		O.append(sum(flow[i]))
		D.append(sum(flow[:,i]))
	return [O, D]

In [5]:
nodeNum = read_excel_data(file_path, "NodeNum")[0][0]
flow = read_excel_data(file_path, "flow")
varCost = read_excel_data(file_path, "varCost")
fixCost = read_excel_data(file_path, "fixCost").flatten()
alpha = read_excel_data(file_path, "alpha")[0][0]
cap = read_excel_data(file_path, "Cap").flatten()
[O, D] = cal_OD()

In [18]:
dna_size = nodeNum-2  # DNA size
cross_rate = 0.85
mutate_rate = 0.35
pop_size = 500
N_GENERATIONS = 500

In [7]:
#resotre tree connectivity
def restore_prufer(prufer_seq):
	# connectivity matrix
	connect_matrix = np.array([[0 for i in range(nodeNum)] for j in range(nodeNum)])

	#list Z
	Z = []
	Y = []
	# Get spokers
	nonHubSet = set([i for i in range(nodeNum)]) - set(prufer_seq)
	#get hubs
	hubSet = set(prufer_seq)
	# To restore connectivity
	restore_seq = [i for i in range(nodeNum)]
	while prufer_seq:
		setPrufer = set(prufer_seq)
		setRestore = set(restore_seq)
		setRest = setRestore - setPrufer
		
		# min value in restore_seq
		minNum = min(setRest)
		restore_seq.remove(minNum)
		#first element in current prufer_seq
		first_ele = prufer_seq.pop(0)

		# set connect_matrix
		connect_matrix[minNum][first_ele] = 1
		connect_matrix[first_ele][minNum] = 1

	connect_matrix[restore_seq[0]][restore_seq[1]] = 1
	connect_matrix[restore_seq[1]][restore_seq[0]] = 1

	# To get Z list, store Z[i, k] value pairs. 
	for i in nonHubSet:
		for k in range(len(connect_matrix[i,:])):
			if connect_matrix[i, k] == 1:
				Z.append([i,k])

	for k in hubSet:
		for m in range(len(connect_matrix[k,:])):
			if (connect_matrix[k,m] == 1) & (m in hubSet):
				Y.append([k,m])
				Y.append([m,k])

	return [connect_matrix, Z.copy(), Y.copy()]

In [8]:
def intraHubFlow(prufer_seq):
	intraHubMatrix = [[0 for i in range(nodeNum)] for j in range(nodeNum)]
	#use package networkx to obtain paths
	tree = nx.from_prufer_sequence(prufer_seq)
	paths = dict(nx.all_pairs_shortest_path(tree))
	for ori in paths:
		for des in paths[ori]:
			e = len(paths[ori][des]) - 1
			s = 0
			if ori not in prufer_seq:
				s += 1
			if des not in prufer_seq:
				e -= 1
			for i in range(s, e):
				path = paths[ori][des]
				#print(path, path[s], path[e])
				intraHubMatrix[path[i]][path[i+1]] += flow[ori][des]
	return intraHubMatrix

In [9]:
def get_Fitness(prufer_seq):
	fix_cost = 0
	var_cost = 0

	#get k matrix
	[connect_matrix, Z, Y] = restore_prufer(prufer_seq[:])
	
	#get intra hub flow matrix to calculate var cost 2
	intraHubMatrix = np.array(intraHubFlow(prufer_seq))

	hubs = set(prufer_seq)

	#fix cost
	for hub in hubs:
		fix_cost += fixCost[hub]
	#print(fix_cost)
	#var cost part 1
	for [i, k] in Z:
		var_cost += (varCost[i][k] * O[i] + varCost[k][i] * D[i])
	#print(var_cost)
	#tmp = var_cost
	#var cost part 2
	cord_x, cord_y = np.nonzero(intraHubMatrix)
	for i in range(len(cord_y)):
		var_cost += alpha * varCost[cord_x[i]][cord_y[i]] * intraHubMatrix[cord_x[i]][cord_y[i]]
	#print(var_cost - tmp)
	return fix_cost + var_cost

In [10]:
# 根据fitness更新选出新一代基因,我们希望fitness越小越好
def select(pop, fitness):
    # convert list to ndarray
    fitness = np.asarray(fitness)
    pop = np.asarray(pop)
    # 当cost越小的时候，我们越倾向于保留这个基因序列
    fitness1 = np.exp(2e8/fitness)
    #idx = np.random.choice(np.arange(pop_size), size=pop_size, replace=True, p=fitness1 / fitness1.sum())
    idx = np.random.choice(np.arange(pop_size), size=pop_size, replace=True, p=fitness1 / fitness1.sum())
    return pop[idx]

In [11]:
def crossover(parent, pop):
    if np.random.rand() < cross_rate:
        i_ = np.random.randint(0, pop_size, size=1)  # select another individual from pop
        # 对于第一个模型，dna_size = 6
        cross_points = np.random.randint(0, 2, dna_size).astype(np.bool)  # choose crossover points
        parent[cross_points] = pop[i_, cross_points]  # mating and produce one child
    return parent

In [12]:
def mutate(child):
    # 对于第一个模型 DNA_size = 6
    for point in range(dna_size):
        if np.random.rand() < mutate_rate:
            swap_point = np.random.randint(0, dna_size)
            swapA, swapB = child[point], child[swap_point]
            child[point], child[swap_point] = swapB, swapA
    return child

In [31]:
# 开始进化,我们采用第二种策略
for i in range(N_GENERATIONS):
    # 进行第一次初始化基因时候
    if(i == 0):
        fitness = []
        # 初始化第一代基因序列
        prufer_seq = [np.random.choice(np.arange(8),6,replace=True) for _ in range(pop_size)]
        # 计算所有的fitness
        for i in prufer_seq:
            buffer = i.tolist()
            fitness.append(get_Fitness(buffer))
            # selection返回一个(500 X 6)ndarray
        pop = select(prufer_seq, fitness)
        pop_copy = pop.copy() 
        for parent in pop:
            # crossover
            child = crossover(parent, pop_copy) 
            # mutation
            child = mutate(child)
            # 孩子变为大人
            parent[:] = child
            
    else:
        fitness = []
        for i in pop:
            buffer = i.tolist()
            fitness.append(get_Fitness(buffer))
            # selection返回一个(500 X 6)ndarray
        pop = select(pop, fitness)
        # 备个份
        pop_copy = pop.copy() 
        for parent in pop:
            # crossover
            child = crossover(parent, pop_copy) 
            # mutation
            child = mutate(child)
            # 孩子变为大人
            parent[:] = child

# 当N_GENERATIONS次之后
fitness = []
for i in pop:
    buffer = i.tolist()
    fitness.append(get_Fitness(buffer))
# 找到此时fitness最小的点
print('minimum:', min(fitness))
index = fitness.index(min(fitness))
print('the distribution is:',pop[index])


KeyboardInterrupt: 

In [13]:
def selcet_MGA(pop, fitness):
    
    return pop

In [14]:
def crossover_MGA(loser_winner):      # crossover for loser
        cross_idx = np.empty((dna_size,)).astype(np.bool)
        for i in range(dna_size):
            cross_idx[i] = True if np.random.rand() < cross_rate else False  # crossover index
        loser_winner[0, cross_idx] = loser_winner[1, cross_idx]  # assign winners genes to loser
        return loser_winner

In [15]:
def mutate_MGA(loser_winner):         # mutation for loser
        mutation_idx = np.empty((dna_size,)).astype(np.bool)
        for i in range(dna_size):
            mutation_idx[i] = True if np.random.rand() < mutate_rate else False  # mutation index
        # flip values in mutation points
        loser_winner[0, mutation_idx] = ~loser_winner[0, mutation_idx].astype(np.bool)
        return loser_winner

In [52]:
def genetic_MGA(prufer_seq,n):
    start_time = datetime.datetime.now()
    # Microbial Genetic Algorithm
    # n = 100
    fitness = []
    # prufer_seq = [np.random.choice(np.arange(nodeNum),dna_size,replace=True) for _ in range(pop_size)]
    pop = np.asarray(prufer_seq)

    for generation in range(N_GENERATIONS):
        for _ in range(n):
            # 随机从整体中取出两个个体
            sub_pop_idx = np.random.choice(np.arange(0, pop_size), size=2, replace=False)
            sub_pop = pop[sub_pop_idx]
            for i in sub_pop:
                buffer = i.tolist()
                fitness.append(get_Fitness(buffer))
            # 比较fitness中两个元素的大小，并排序，记录各自的序列
            fitness = np.asarray(fitness)
            fitness1 = np.exp(2e8/fitness)
            loser_winner_idx = np.argsort(fitness1)
            loser_winner = sub_pop[loser_winner_idx]
            loser_winner = crossover_MGA(loser_winner)
            loser_winner = mutate_MGA(loser_winner)
            pop[sub_pop_idx] = loser_winner
            fitness = []

    # 当N_GENERATIONS次之后
    fitness = []
    for i in pop:
        buffer = i.tolist()
        fitness.append(get_Fitness(buffer))
    # 找到此时fitness最小的点
    print('minimum:', min(fitness))
    index = fitness.index(min(fitness))
    # print('the distribution is:',pop[index])

    end_time = datetime.datetime.now()
    interval = (end_time-start_time).seconds
    # print ('Seconds:\t', interval)
    return (min(fitness))

In [57]:
cross_rate = 0.85
mutate_rate = 0.1
pop_size = 10000
N_GENERATIONS = 500
n = 35
import datetime

In [ ]:
global_Max = 0
loops = 0
for i in range(100):
    loops += 1
    prufer_seq = [np.random.choice(np.arange(nodeNum),dna_size,replace=True) for _ in range(pop_size)]
    fitness = genetic_MGA(prufer_seq,n)
    if(fitness == 6482028.75):
        global_Max += 1
rate = global_Max/loops
print("convergence index is:",rate)

In [49]:
np.arange(nodeNum)
[np.random.choice(np.arange(nodeNum),dna_size,replace=True) for _ in range(pop_size)]

array([0, 1, 2, 3, 4, 5, 6, 7])

In [88]:
print('全局最优解 6482028，75')

全局最优解 6482028，75
